In [2]:
import os
import sys
import math
import json
import shutil
import subprocess
from glob import glob
from datetime import datetime
from pathlib import Path
from tqdm import tqdm
import time
import warnings
import numpy as np
import pandas as pd
import librosa
import librosa.display

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model


from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
print("checkeo de librerias completOo")


checkeo de librerias completOo


In [2]:
# ------------------------
# ---- VARIABLES ---------
# ------------------------
DATA_DIR = r"D:\dataset pf\COPIA\organized_by_type\FILTERED_AUDIO_10s\data_augmentation"
OUTPUT_ROOT = r"D:\dataset pf\COPIA\organized_by_type\FILTERED_AUDIO_10s\data_augmentation_train2"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

# Audio / features
SR = 4000
DURATION = 10
SAMPLES = SR * DURATION
N_MFCC = 40
N_MELS = 40
HOP_LENGTH = 256

# Entrenamiento / CV
N_FOLDS = 5
EPOCHS = 60
BATCH_SIZE = 32
RANDOM_STATE = 42

# Modelo
LEARNING_RATE = 1e-3
DROPOUT = 0.25

# Cache
CACHE_DIR = os.path.join(OUTPUT_ROOT, 'feature_cache')
os.makedirs(CACHE_DIR, exist_ok=True)

FEATURE_TYPES = ['mfcc', 'mel', 'concat']  # tipos de features

# GPU
physical_gpus = tf.config.list_physical_devices('GPU')
if physical_gpus:
    try:
        try:
            out = subprocess.check_output(['nvidia-smi', '--query-gpu=memory.total', '--format=csv,noheader,nounits'])
            total_mb = int(out.decode('utf-8').strip().split('\n')[0])
            limit_mb = int(total_mb * 0.85)
            tf.config.experimental.set_virtual_device_configuration(
                physical_gpus[0],
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=limit_mb)])
            print(f" GPU configurada con límite {limit_mb} MB")
        except Exception:
            tf.config.experimental.set_memory_growth(physical_gpus[0], True)
            print(" nvidia-smi no disponible: activado memory_growth")
    except Exception as e:
        print(' No se pudo configurar GPU:', e)
else:
    print(' No se detectó GPU: ejecutando en CPU')

print("check")

 GPU configurada con límite 3481 MB
check


In [4]:
#recorrido de carpetas
def list_files_and_labels(root_dir):
    filepaths, labels = [], []
    classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
    for cls in classes:
        cls_dir = os.path.join(root_dir, cls)
        for ext in ('*.wav', '*.WAV'):
            for f in glob(os.path.join(cls_dir, ext)):
                filepaths.append(f)
                labels.append(cls)
    return filepaths, labels

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)
    return path

print("se crearon las funciones correctamenteE")

se crearon las funciones correctamenteE


In [5]:
#extraccion de caracteristicas
def extract_mfcc(filepath, sr=SR, n_mfcc=N_MFCC, hop_length=HOP_LENGTH, duration=DURATION):
    y, _ = librosa.load(filepath, sr=sr, duration=duration)
    if len(y) < SAMPLES: y = np.pad(y, (0, SAMPLES - len(y)))
    else: y = y[:SAMPLES]
    return librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length)

def extract_mel(filepath, sr=SR, n_mels=N_MELS, hop_length=HOP_LENGTH, duration=DURATION):
    y, _ = librosa.load(filepath, sr=sr, duration=duration)
    if len(y) < SAMPLES: y = np.pad(y, (0, SAMPLES - len(y)))
    else: y = y[:SAMPLES]
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, hop_length=hop_length)
    return librosa.power_to_db(mel, ref=np.max)

def pad_feature(feat, target_frames):
    if feat.shape[1] < target_frames:
        pad_width = target_frames - feat.shape[1]
        feat = np.pad(feat, ((0,0),(0,pad_width)), mode='constant')
    else:
        feat = feat[:, :target_frames]
    return feat

print("se crearon las funciones correctamente")

se crearon las funciones correctamente


In [6]:
# Listado completo, SEPARAR EN 95/5%
filepaths, labels = list_files_and_labels(DATA_DIR)
le = LabelEncoder()
y_enc = le.fit_transform(labels)
class_names = list(le.classes_)

# Split 95%-5%
X_train_files, X_test_web_files, y_train_labels, y_test_web_labels = train_test_split(
    filepaths, labels, test_size=0.05, stratify=labels, random_state=RANDOM_STATE
)

# Crear carpeta test_web
test_web_dir = os.path.join(OUTPUT_ROOT, 'test_web')
os.makedirs(test_web_dir, exist_ok=True)

# Copiar audios de prueba web
for fp, lbl in zip(X_test_web_files, y_test_web_labels):
    class_dir = os.path.join(test_web_dir, lbl)
    os.makedirs(class_dir, exist_ok=True)
    shutil.copy2(fp, class_dir)

print(f" {len(X_test_web_files)} audios copiados a {test_web_dir} para prueba web")


 1423 audios copiados a D:\dataset pf\COPIA\organized_by_type\FILTERED_AUDIO_10s\data_augmentation_train2\test_web para prueba web


In [7]:


# Parámetro: cuántas partes crear (ajústalo según memoria disponible)
N_PARTS = 2  # <-- cambia este valor según tus recursos

# ejemplo y TARGET_FRAMES ya definidos antes
_example_mfcc = extract_mfcc(X_train_files[0])
TARGET_FRAMES = _example_mfcc.shape[1]

cached_paths = {}
for ftype in FEATURE_TYPES:
    # base path (ya que ahora habrá varias partes)
    cp_base = os.path.join(CACHE_DIR, f'features_{ftype}')
    # lista de posibles archivos por parte
    part_paths = [f"{cp_base}_part{idx+1}.npz" for idx in range(N_PARTS)]
    # guardaremos en cached_paths info útil (manteniendo la variable cached_paths)
    cached_paths[ftype] = {'path': cp_base, 'parts': part_paths, 'X': None, 'y': None}

# función auxiliar para cargar/concatenar partes sólo cuando se necesite
def load_cached(ftype, concat=True):
    """
    Carga las partes del cache para 'ftype'.
    Si concat==True retorna (X,y) concatenados (y en int encoded).
    Si concat==False retorna listas (X_parts, y_parts) con cada parte por separado.
    Además setea cached_paths[ftype]['X'] y ['y'] si concat=True.
    """
    info = cached_paths[ftype]
    parts = info['parts']
    X_parts = []
    y_parts = []
    any_exist = False

    for p in parts:
        if os.path.exists(p):
            any_exist = True
            data = np.load(p, allow_pickle=True)
            Xp, yp = data['X'], data['y']
            X_parts.append(Xp)
            y_parts.append(yp)
        else:
            continue

    if not any_exist:
        return None if concat else ([], [])

    if concat:
        X = np.concatenate(X_parts, axis=0)
        y = np.concatenate(y_parts, axis=0)
        cached_paths[ftype]['X'] = X
        cached_paths[ftype]['y'] = y
        return X, y
    else:
        return X_parts, y_parts

# Procesar y crear partes (si no existen ya)
for ftype in FEATURE_TYPES:
    part_paths = cached_paths[ftype]['parts']
    meta_path = cached_paths[ftype]['path'] + "_metadata.json"

    if any(os.path.exists(p) for p in part_paths):
        print(f'→ Cargando cache (partes) de {ftype} desde {cached_paths[ftype]["path"]} (partes detectadas)')
    else:
        print(f'→ Calculando {ftype} para {len(X_train_files)} archivos y guardando en {N_PARTS} partes...')
        indices = np.arange(len(X_train_files))
        splits = np.array_split(indices, N_PARTS)

        le_local = LabelEncoder()
        all_labels = [lbl for lbl in y_train_labels]
        le_local.fit(all_labels)

        total_samples = 0
        for part_idx, idx_chunk in enumerate(splits):
            feats = []
            labels_part = []
            for i in idx_chunk:
                fp = X_train_files[i]
                lbl = y_train_labels[i]

                if ftype == 'mfcc':
                    f = extract_mfcc(fp)
                elif ftype == 'mel':
                    f = extract_mel(fp)
                elif ftype == 'concat':
                    f = np.vstack([extract_mfcc(fp), extract_mel(fp)])
                else:
                    raise ValueError(f"Feature type desconocido: {ftype}")

                f = pad_feature(f, TARGET_FRAMES)
                feats.append(f)
                labels_part.append(lbl)

            X_part = np.array(feats)[..., np.newaxis]
            y_part = np.array(le_local.transform(labels_part))

            save_path = part_paths[part_idx]
            np.savez_compressed(save_path, X=X_part, y=y_part)
            total_samples += X_part.shape[0]
            print(f'  → Guardada parte {part_idx+1}/{N_PARTS} en {save_path} (X.shape={X_part.shape})')

        # Guardar metadata asociada al cache
        metadata = {
            "feature_type": ftype,
            "base_path": cached_paths[ftype]['path'],
            "num_parts": N_PARTS,
            "parts": part_paths,
            "num_samples": total_samples,
            "classes": list(le_local.classes_)
        }

        with open(meta_path, "w", encoding="utf-8") as f:
            json.dump(metadata, f, indent=4, ensure_ascii=False)

        print(f'→ Guardado cache en partes y metadata para {ftype} en {meta_path}')

    print("cargado cacheo")


→ Calculando mfcc para 27025 archivos y guardando en 2 partes...
  → Guardada parte 1/2 en D:\dataset pf\COPIA\organized_by_type\FILTERED_AUDIO_10s\data_augmentation_train2\feature_cache\features_mfcc_part1.npz (X.shape=(13513, 40, 157, 1))
  → Guardada parte 2/2 en D:\dataset pf\COPIA\organized_by_type\FILTERED_AUDIO_10s\data_augmentation_train2\feature_cache\features_mfcc_part2.npz (X.shape=(13512, 40, 157, 1))
→ Guardado cache en partes y metadata para mfcc en D:\dataset pf\COPIA\organized_by_type\FILTERED_AUDIO_10s\data_augmentation_train2\feature_cache\features_mfcc_metadata.json
cargado cacheo
→ Calculando mel para 27025 archivos y guardando en 2 partes...
  → Guardada parte 1/2 en D:\dataset pf\COPIA\organized_by_type\FILTERED_AUDIO_10s\data_augmentation_train2\feature_cache\features_mel_part1.npz (X.shape=(13513, 40, 157, 1))
  → Guardada parte 2/2 en D:\dataset pf\COPIA\organized_by_type\FILTERED_AUDIO_10s\data_augmentation_train2\feature_cache\features_mel_part2.npz (X.shape=

In [ ]:
# modelo de CCN
def build_cnn(input_shape, num_classes, dropout=DROPOUT, lr=LEARNING_RATE):
    model = Sequential([
        Conv2D(16, (3,3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Dropout(dropout),

        Conv2D(32, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Dropout(dropout),

        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Dropout(dropout),

        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Dropout(dropout),

        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(dropout),
        Dense(len(class_names), activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=lr), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model
print("funcion cargada correctamente")

In [ ]:
results_summary = {}
global_start = time.time()

# reproducibilidad (mantener la variable RANDOM_STATE si ya existe)
SEED = RANDOM_STATE if 'RANDOM_STATE' in globals() else 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# silenciar warnings molestos
warnings.filterwarnings('ignore')

# Definir número de clases antes del loop (asumimos `le` existe en el entorno como antes)
class_names = list(le.classes_)
num_classes = len(class_names)

for ftype in FEATURE_TYPES:
    print('\n' + '='*80)
    print(f'  ENTRENANDO FEATURE TYPE: {ftype.upper()} ')
    print('='*80)
    
    feature_start = time.time()
    
    entry = cached_paths[ftype]
    X_all, y_all = entry['X'], entry['y']
    
    # Crear carpetas por feature type (manteniendo nombres existentes)
    out_ft = os.path.join(OUTPUT_ROOT, ftype)
    models_dir = ensure_dir(os.path.join(out_ft, 'models'))
    reports_dir = ensure_dir(os.path.join(out_ft, 'reports'))
    histories_dir = ensure_dir(os.path.join(out_ft, 'histories'))
    cm_dir = ensure_dir(os.path.join(out_ft, 'confusion_matrices'))
    curves_dir = ensure_dir(os.path.join(out_ft, 'learning_curves'))
    
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    fold_metrics = []
    
    # Mostrar arquitectura antes de los folds (ejemplo)
    try:
        model_example = build_cnn(X_all.shape[1:], num_classes)
        print("\n🔹 Arquitectura de la CNN para feature type:", ftype)
        model_example.summary()
        plot_model(model_example, to_file=os.path.join(out_ft, f'{ftype}_CNN_architecture.png'),
                   show_shapes=True, show_layer_names=True)
    except Exception as e:
        print(f"⚠️ No se pudo construir/guardar arquitectura de ejemplo para {ftype}: {e}")
    
    print(f'Feature Type "{ftype}" tiene {X_all.shape[0]} muestras con {num_classes} clases.')
    
    for fold, (train_idx, test_idx) in tqdm(
        enumerate(skf.split(X_all, y_all), 1),
        total=N_FOLDS,
        desc=f'Training {ftype.upper()}',
        unit='fold',
        ncols=100
    ):
        print(f'\n🔹 Fold {fold}/{N_FOLDS}')
        fold_start = time.time()
        
        try:
            X_train, X_test = X_all[train_idx], X_all[test_idx]
            y_train, y_test = y_all[train_idx], y_all[test_idx]
            
            # Normalización per-dataset (manteniendo variable names)
            X_train, X_test = X_train.astype('float32'), X_test.astype('float32')
            mean, std = X_train.mean(), X_train.std() + 1e-9
            X_train, X_test = (X_train - mean)/std, (X_test - mean)/std
            
            model = build_cnn(X_train.shape[1:], num_classes)
            modelfn = os.path.join(models_dir, f'{ftype}_fold{fold}_best.h5')
            
            callbacks = [
                ModelCheckpoint(modelfn, monitor='val_accuracy', save_best_only=True, verbose=1),
                EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
                ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
            ]
            
            print(f'Entrenando modelo para "{ftype}" - Fold {fold}...')
            hist_start = time.time()
            history = model.fit(
                X_train, y_train,
                validation_data=(X_test, y_test),
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                callbacks=callbacks,
                verbose=1
            )
            hist_time = time.time() - hist_start
            print(f'Tiempo de entrenamiento fold {fold}: {hist_time:.2f}s ({hist_time/60:.2f} min)')
            
            # Guardar historial por fold
            hist_df = pd.DataFrame(history.history)
            hist_df.to_csv(os.path.join(histories_dir, f'{ftype}_fold{fold}_history.csv'), index=False)
            
            # Evaluación usando el mejor modelo guardado (si existe)
            if os.path.exists(modelfn):
                best_model = tf.keras.models.load_model(modelfn)
            else:
                print("⚠️ Modelo guardado no encontrado, usando modelo actual en memoria.")
                best_model = model
            
            preds = np.argmax(best_model.predict(X_test, verbose=0), axis=1)
            
            acc = float(accuracy_score(y_test, preds))
            f1 = float(f1_score(y_test, preds, average='macro'))
            prec = float(precision_score(y_test, preds, average='macro', zero_division=0))
            rec = float(recall_score(y_test, preds, average='macro', zero_division=0))
            
            print(f'Resultados fold {fold}: acc={acc:.4f} f1={f1:.4f} prec={prec:.4f} rec={rec:.4f}')
            print(f'Tiempo total fold {fold}: {time.time() - fold_start:.2f}s ({(time.time()-fold_start)/60:.2f} min)')
            
            # Reporte de clasificación y matriz confusión
            class_rep = classification_report(y_test, preds, target_names=class_names, output_dict=True)
            with open(os.path.join(reports_dir, f'{ftype}_fold{fold}_classification.json'), 'w') as fh:
                json.dump(class_rep, fh, indent=2)
            
            cm = confusion_matrix(y_test, preds)
            plt.figure(figsize=(10,8))
            sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
            plt.title(f'Confusion Matrix - {ftype} Fold {fold}')
            plt.ylabel('True')
            plt.xlabel('Pred')
            plt.tight_layout()
            plt.savefig(os.path.join(cm_dir, f'{ftype}_fold{fold}_cm.png'))
            plt.close()
            
            fold_metrics.append({'fold': fold, 'acc': acc, 'f1': f1, 'prec': prec, 'rec': rec})
        
        except Exception as err_fold:
            # Si un fold falla, lo registramos y continuamos con los siguientes
            print(f"❗ Error en fold {fold} de {ftype}: {err_fold}")
            # Guardar registro de error como archivo para debug
            with open(os.path.join(out_ft, f'fold{fold}_error.txt'), 'w') as fh:
                fh.write(str(err_fold))
            continue
    
    # Estadísticas finales por feature
    dfm = pd.DataFrame(fold_metrics)
    if dfm.shape[0] > 0:
        summary = {
            'feature_type': ftype,
            'mean_acc': float(dfm['acc'].mean()),
            'std_acc': float(dfm['acc'].std(ddof=0)),   # ddof=0 para poblacional; cambia si prefieres ddof=1
            'mean_f1_macro': float(dfm['f1'].mean()),
            'std_f1_macro': float(dfm['f1'].std(ddof=0)),
            'per_fold': fold_metrics
        }
    else:
        summary = {
            'feature_type': ftype,
            'mean_acc': None,
            'std_acc': None,
            'mean_f1_macro': None,
            'std_f1_macro': None,
            'per_fold': fold_metrics
        }
    
    # Curvas de aprendizaje promedio (accuracy + loss)
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    for fold_idx in range(N_FOLDS):
        hist_file = os.path.join(histories_dir, f'{ftype}_fold{fold_idx+1}_history.csv')
        if os.path.exists(hist_file):
            h = pd.read_csv(hist_file)
            if 'accuracy' in h.columns:
                plt.plot(h['accuracy'], label=f'Fold {fold_idx+1}')
            elif 'acc' in h.columns:
                plt.plot(h['acc'], label=f'Fold {fold_idx+1}')
    plt.title(f'Accuracy por fold - {ftype}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1,2,2)
    for fold_idx in range(N_FOLDS):
        hist_file = os.path.join(histories_dir, f'{ftype}_fold{fold_idx+1}_history.csv')
        if os.path.exists(hist_file):
            h = pd.read_csv(hist_file)
            if 'loss' in h.columns:
                plt.plot(h['loss'], label=f'Fold {fold_idx+1}')
    plt.title(f'Loss por fold - {ftype}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(curves_dir, f'{ftype}_learning_curves.png'))
    plt.close()
    
    results_summary[ftype] = summary
    dfm.to_csv(os.path.join(out_ft, 'fold_metrics.csv'), index=False)
    with open(os.path.join(out_ft, 'summary.json'), 'w') as fh:
        json.dump(summary, fh, indent=2)
    
    total_feature_time = time.time() - feature_start
    print(f'\n Feature "{ftype}" completado en {total_feature_time:.2f}s ({total_feature_time/60:.2f} min)\n')

# Comparación general y guardado
comp_df = pd.DataFrame([
    {'feature': k,
     'mean_acc': v.get('mean_acc', None),
     'std_acc': v.get('std_acc', None),
     'mean_f1_macro': v.get('mean_f1_macro', None),
     'std_f1_macro': v.get('std_f1_macro', None)}
    for k,v in results_summary.items()
])
comp_df.to_csv(os.path.join(OUTPUT_ROOT, 'comparison_summary.csv'), index=False)

total_time = time.time() - global_start
print(f"\nENTRENAMIENTO COMPLETADO en {total_time:.2f}s ({total_time/60:.2f} min)")
print("Resultados guardados en:", OUTPUT_ROOT)

# --- IMPRESIÓN FINAL: media y desviación estándar de los k-folds por feature ---
print("\n=== RESUMEN FINAL (accuracy por feature) ===")
for ftype, info in results_summary.items():
    mean_acc = info.get('mean_acc')
    std_acc = info.get('std_acc')
    if mean_acc is not None:
        print(f'- {ftype}: mean_acc = {mean_acc:.4f}, std_acc = {std_acc:.4f}')
    else:
        print(f'- {ftype}: No se obtuvieron folds válidos.')

# --- GRAFICO COMPARATIVO ENTRE LAS 3 EXTRACCIONES (mfcc, mel, concat) ---
# Filtrar sólo las 3 que pidió (si están presentes)
features_to_plot = [f for f in ['mfcc', 'mel', 'concat'] if f in results_summary]
if len(features_to_plot) == 0:
    print("⚠️ Ninguna de las features 'mfcc', 'mel' o 'concat' está presente en results_summary. No se generará el gráfico comparativo.")
else:
    means = [results_summary[f]['mean_acc'] for f in features_to_plot]
    stds = [results_summary[f]['std_acc'] for f in features_to_plot]
    
    plt.figure(figsize=(8,6))
    x_pos = np.arange(len(features_to_plot))
    plt.bar(x_pos, means, yerr=stds, capsize=8)
    plt.xticks(x_pos, features_to_plot)
    plt.ylabel('Mean Accuracy')
    plt.xlabel('Feature Type')
    plt.title('Comparación: Mean Accuracy ± Std (k-folds)')
    for i, v in enumerate(means):
        plt.text(i, v + 0.005, f'{v:.3f}', ha='center', va='bottom')
    plt.tight_layout()
    comparison_path = os.path.join(OUTPUT_ROOT, 'feature_mean_comparison_mfcc_mel_concat.png')
    plt.savefig(comparison_path)
    plt.close()
    print(f"Gráfico comparativo guardado en: {comparison_path}")

# Guardar summary completo de results_summary
with open(os.path.join(OUTPUT_ROOT, 'results_summary_all.json'), 'w') as fh:
    json.dump(results_summary, fh, indent=2)